In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 108.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.8 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/CS182/CS\ 182\ Final\ Project/Model

/content/drive/.shortcut-targets-by-id/1hzhdcGA40OipfzF0SRT7omKvzCSi0q4r/CS 182 Final Project/Model


In [ ]:
import os
import torch
from datasets import load_dataset
from datasets import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel, get_peft_model, PrefixTuningConfig
from trl import SFTTrainer
import pandas as pd

In [ ]:
from datasets import load_dataset
import re

def read_files(file_names: list):
  dfs = []
  for file in file_names:
    dfs.append(pd.read_csv(f"./../Data Processing/Training data/{file}"))
  df = pd.concat(dfs)
  df.columns = ["q", "a"]
  return df

train_df = read_files(["126.csv", "127.csv", "182.csv", "189_fa.csv", "189_sp.csv"])
dataset = Dataset.from_pandas(train_df)
dataset = dataset.train_test_split(test_size=0.2, seed=10)
train_dataset, val_dataset = dataset["train"], dataset["test"]

# Load the dataset
# dataset_name = 'flax-sentence-embeddings/stackexchange_math_jsonl'
# dataset = load_dataset(dataset_name, name='titlebody_answer')
q_col = 'q'
a_col = 'a'

# Define a function to transform the data
def transform_conversation(example):
  question = example[q_col]
  answer = example[a_col]
  return {'text': f'<s>[INST] {question} [/INST] {answer} </s>'}

# Apply the transformation
train_dataset = train_dataset.map(transform_conversation).select_columns('text')
val_dataset = val_dataset.map(transform_conversation).select_columns('text')

Map:   0%|          | 0/984 [00:00<?, ? examples/s]

Map:   0%|          | 0/247 [00:00<?, ? examples/s]

In [ ]:
%cd Geovanni

/content/drive/.shortcut-targets-by-id/1hzhdcGA40OipfzF0SRT7omKvzCSi0q4r/CS 182 Final Project/Model/Geovanni


In [ ]:
# The model that you want to train from the Hugging Face hub
model_name = "gpt2-medium"

# Fine-tuned model name
new_model = "gpt2_lora_r64_a256_epoch10"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 256

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results15"

# Number of training epochs
num_train_epochs = 10

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule (constant a bit better than cosine)
lr_scheduler_type = "constant"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 200

# Log every X updates steps
logging_steps = 50

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [ ]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# # # Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard",
    evaluation_strategy="steps",
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model)

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at gpt2-medium and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/984 [00:00<?, ? examples/s]

Map:   0%|          | 0/247 [00:00<?, ? examples/s]

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
50,3.155800,2.378906
100,2.546200,2.201172
150,2.500300,2.160156
200,2.387200,2.123047
250,2.299300,2.167969
300,2.325300,2.123047
350,2.229500,2.117188
400,2.280000,2.089844
450,2.257300,2.080078
500,2.205800,2.083984


In [ ]:
%reload_ext tensorboard
%tensorboard --logdir results2/runs

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
2023-11-27 09:53:01.032195: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-27 09:53:01.032284: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-27 09:53:01.032323: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-27 09:53:02.671641: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorf

In [ ]:
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "Define the cross attention mechanism in the context of encoder-decoder models and explain how it differs from self-attention. What is the source of the queries and the key-value pairs in the cross attention mechanism?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


<s>[INST] Define the cross attention mechanism in the context of encoder-decoder models and explain how it differs from self-attention. What is the source of the queries and the key-value pairs in the cross attention mechanism? [/INST] The cross attention mechanism is a technique in encoder-decoder models where the decoder recognizes more than one source, and the decoded decoder can refer to more than one source, each independently decoded. The source of queries has to be acknowleged, which also has to be acknowleged, as the decoder must be aware of the queries that are in advance of the decoder. The key-value pairs, however, must first be independently decoded in order to detect queries and key pairs, as well as the keys and values that occur. </s>

 </s> </s>

 The cross attention mechanism is different from self-attention in that it features multiple channels, each corresponding


In [ ]:
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "Define the cross attention mechanism in the context of encoder-decoder models and explain how it differs from self-attention. What is the source of the queries and the key-value pairs in the cross attention mechanism?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


<s>[INST] Define the cross attention mechanism in the context of encoder-decoder models and explain how it differs from self-attention. What is the source of the queries and the key-value pairs in the cross attention mechanism? [/INST] The cross attention mechanism is used in the encoder-decoder model to identify the subsequence that has to be returned by the decoders during the transition between transitions. In the Cross Attention Mechanism, all the queries and key-value pairs are replaced with the subsequences of the subsequences which have to been returned by the decoders. This is different than self-attention, where the cross attention queries and key-value pairs occur in the entire segment prior to the transition.

 </s>

 </s> For cross attention, assuming that the segments of the encoder-decoder model have been removed߅ޅޅޅޅޅޅޅޅ�


In [ ]:
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "Define Shannon's Source Coding Theorem and its relevance to the concept of message compression in digital communication systems. How does this theorem relate to the first step mentioned in the basic steps of a digital communication system?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] Define Shannon's Source Coding Theorem and its relevance to the concept of message compression in digital communication systems. How does this theorem relate to the first step mentioned in the basic steps of a digital communication system? [/INST] The Shannon's Source Coding Theorem states that if a message compression procedure is useful and can satisfy the requirement, then it is also responsible but the message isn't compressed. This theorem is relevant since messages are compressible by a signal header, therefore it allows us to conclude that whatever signal is in the header is also a signal along the horizontal direction of the system, thereby ensuring that the system gets a signal compression rate.

 </s>The first step mentioned in the basic steps of a digital communications system is to compress the message to a format that will satisfy the compression formula. This is related to Shannon's Source Coding Theorem, where it states that the first step is another signal and

In [ ]:
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "What do you cheese that isn't yours?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] What do you cheese that isn't yours? [/INST] cheese that isn't yours exists in the context of the subsequence defined below. It doesn't mean that the subsequence describing the neighborship of the cheesemessage depends on that subsequence, but it does indicate that it is feasible to cheese that subsequence.
</s> The cheesemessage $p = [p neighb=g] && g \geq $r^1$ for $g = #n$. The cheesemessage $p$ is $p neighb=g$, and the neighborship $q$ is the neighborship of the cheesemessage $p$ with the cheesemessage $g$. This cheese is feasible because $q$. </s>

 </s> </s> In the context of cheesemotessages, cheese cheesecks can be substituted for the subsequences without violating the subsequence specifications. Cheesemotessages exist because they


In [ ]:
0

<s>[INST] Define Shannon's Source Coding Theorem and its relevance to the concept of message compression in digital communication systems. How does this theorem relate to the first step mentioned in the basic steps of a digital communication system? [/INST]

Shannon's proof: Shannon's proof for the basic steps of a digital communication system. What is Shannon's proof for the basic steps of a digital communication system? You can find Shannon's results in my presentation of his proof for the basic steps of a digital communication system in "The basic steps of a digital communication system", and I also add the proofs to this article of mine "Shannon's Proof: A Basic Proof of The Basic Steps of A Digital Communication System":

Shannon's proof:

Let I know that Shannon's proof for the basic steps of a digital communication system consists of a two-factor authentication, (1) this is verified here from here, (2) if she did not know all